# Importando Paquetes

In [2]:
import os
import pandas as pd
import numpy as np
import seaborn
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
pd.options.plotting.backend = "plotly"
import warnings
warnings.filterwarnings('ignore')

# Lectura Dataset

In [3]:
earthquakes = pd.read_csv('earthquakes-2021-01-19_21-44-08_-0500.tsv', sep = '\t')
earthquakes.shape

(6217, 39)

# Data Cleaning 

In [4]:
earthquakes = earthquakes[earthquakes['Search Parameters'].isnull()]
earthquakes = earthquakes.drop('Search Parameters', axis = 1)
earthquakes['Decade'] = (earthquakes['Year'] / 10).astype('int')
earthquakes['Century'] = (earthquakes['Year'] // 100).astype('int') + 1
earthquake_21 = earthquakes.query('Century >= 20')
earthquake_21['Country'] = earthquake_21['Location Name'].str.split(':', expand = True)[0]
earthquake_21 =earthquake_21[earthquake_21['Latitude'].notna()]
earthquake_21=earthquake_21[earthquake_21['Longitude'].notna()]



In [5]:
earthquake_21[["Latitude","Longitude","Country"]]

,Latitude,Longitude,Country
2476,-0.030,127.250,INDONESIA
2477,36.500,133.500,JAPAN
2478,-6.840,106.960,INDONESIA
2479,20.000,-105.000,MEXICO
2480,20.000,-105.000,MEXICO
...,...,...,...
6212,54.608,-159.655,ALASKA
6213,37.918,26.790,GREECE
6214,-39.343,-74.990,CHILE
6215,45.422,16.255,BALKANS NW


In [6]:
earthquake_21.columns

Index(['Year', 'Mo', 'Dy', 'Hr', 'Mn', 'Sec', 'Tsu', 'Vol', 'Location Name',
       'Latitude', 'Longitude', 'Focal Depth (km)', 'Mag', 'MMI Int', 'Deaths',
       'Death Description', 'Missing', 'Missing Description', 'Injuries',
       'Injuries Description', 'Damage ($Mil)', 'Damage Description',
       'Houses Destroyed', 'Houses Destroyed Description', 'Houses Damaged',
       'Houses Damaged Description', 'Total Deaths', 'Total Death Description',
       'Total Missing', 'Total Missing Description', 'Total Injuries',
       'Total Injuries Description', 'Total Damage ($Mil)',
       'Total Damage Description', 'Total Houses Destroyed',
       'Total Houses Destroyed Description', 'Total Houses Damaged',
       'Total Houses Damaged Description', 'Decade', 'Century', 'Country'],
      dtype='object')

Se observa que no existe una normalizaciíon en la descripción de los paises

In [7]:
earthquake_21.Country.unique()

array(['INDONESIA', 'JAPAN', 'MEXICO', 'VENEZUELA', 'COSTA RICA',
       'TURKEY', 'SOLOMON ISLANDS', 'ALASKA', 'PAPUA NEW GUINEA',
       'COLOMBIA', 'RUSSIA', 'ECUADOR', 'CHINA', 'CALIFORNIA', 'BULGARIA',
       'IWATE, JAPAN', 'NEW CALEDONIA', 'PHILIPPINES', 'NICARAGUA',
       'GUATEMALA', 'TONGA ISLANDS', 'AZERBAIJAN',
       'EL SALVADOR-GUATEMALA', 'IRAN', 'GUAM', 'UTAH', 'UZBEKISTAN',
       'MONGOLIA', 'ISRAEL', 'VANUATU ISLANDS', 'ITALY', 'GREECE',
       'CHILE', 'PANAMA-COLOMBIA', 'TAIWAN', 'NEW ZEALAND', 'PANAMA',
       'KERMADEC ISLANDS', 'VANUATU', 'INDIA', 'ALBANIA', 'KIRIBATI',
       'GEORGIA', 'SYRIA', 'TUNISIA', 'COLOMBIA-ECUADOR', 'SAINT LUCIA',
       'ETHIOPIA', 'AUSTRALIA', 'GHANA', 'MARTINIQUE', 'PERU', 'JAMAICA',
       'TAJIKISTAN', 'SW. SUMATRA', 'HAWAII', 'AFGHANISTAN',
       'MYANMAR (BURMA)', 'MOROCCO', 'FIJI ISLANDS', 'PORTUGAL',
       'CAMEROON', 'FRANCE', 'PAKISTAN', 'HONDURAS', 'ALGERIA', 'ZAMBIA',
       'KAZAKHSTAN', 'GREECE-ALBANIA', 'MICRONESIA

# Generación Label Pais a través de Longitud y Latitud

In [9]:
import reverse_geocoder as rg
from iso3166 import countries


In [ ]:
earthquake_21["lblctry"] = earthquake_21.apply(lambda x:    dict(rg.search ((x.Latitude,x.Longitude))[0])['cc'], axis=1 )
earthquake_21["ISO_APLHA"] = earthquake_21.apply(lambda x:    countries.get(x.lblctry)[2] , axis=1)
erthquake_21["Country"] = earthquake_21.apply(lambda x:    countries.get(x.lblctry)[4] , axis=1)

In [ ]:
earthquake_21.to_csv('earthquakes-prec.csv')

In [12]:
earthquake_21=pd.read_csv('earthquakes-prec.csv',sep=',')

# Ploteo 

In [48]:
import plotly.express as px
px.set_mapbox_access_token("pk.eyJ1Ijoiam9zZWJsZXgiLCJhIjoiY2trZ3U3MjBpMDk1ODJxczdsY3o4eHlociJ9.aIZv4zfwVt3XcUCiIefVVQ")

fig = px.scatter_mapbox(earthquake_21[earthquake_21["Mag"].notna()], lat="Latitude", lon="Longitude",size="Mag",color="Mag"
                  ,color_continuous_scale=px.colors.cyclical.IceFire, size_max=12
                        
                        , zoom=1)
fig.show()

In [50]:

def generate_heatmap_sum(variable):
    df=earthquake_21.get(['ISO_APLHA','Country',variable]).groupby(['ISO_APLHA','Country']).sum().reset_index()
    fig = px.choropleth(df, locations="ISO_APLHA",
                        color=variable, # lifeExp is a column of gapminder
                        hover_name="Country", # column to add to hover information
                        color_continuous_scale=px.colors.sequential.Cividis_r)
    fig.show()

In [70]:
generate_heatmap_sum('Damage ($Mil)')#Injuries Damage Deaths

In [43]:

def generate_heatmap_mean(variable):
    df=earthquake_21.get(['ISO_APLHA','Country',variable]).groupby(['ISO_APLHA','Country']).mean().reset_index()
    fig = px.choropleth(df, locations="ISO_APLHA",
                        color=variable, # lifeExp is a column of gapminder
                        hover_name="Country", # column to add to hover information
                        color_continuous_scale=px.colors.cyclical.IceFire)
    fig.show()

In [55]:
generate_heatmap_mean('Mag')